## 파일 경로 설정 

In [1]:
!mkdir -p ~/jungcheck    
!pwd    
!cd ~/jungcheck    
!mkdir -p ./data/google_review    
!mkdir -p ./data/naver_review  

/home/aiffel-dj19/jungcheck/DataPreprocessing


In [2]:
# 현재 경로 확인 
!pwd

/home/aiffel-dj19/jungcheck/DataPreprocessing


아래 tree 명령어로 상위 폴더(jungcheck)내 파일 구성을 볼 수 있습니다.    

In [3]:
# jungcheck 폴더내 경로 확인 
!tree -L 3 -N .

.
├── data
│   ├── google_review
│   │   ├── 한빛탑.csv
│   │   ├── 엑스포다리.csv
│   │   ├── 엑스포음악분수.csv
│   │   ├── 엑스포과학공원.csv
│   │   └── 세계엑스포기념품박물관.csv
│   └── naver_review
│       ├── 오월드 관광 대전.csv
│       ├── 오월드 여행 후기.csv
│       ├── 오월드 여행 대전.csv
│       ├── 오월드 관광 주말에.csv
│       ├── 오월드 여행 주말에.csv
│       ├── 뿌리공원 여행 후기.csv
│       ├── 뿌리공원 관광 후기.csv
│       ├── 뿌리공원 관광 대전.csv
│       ├── 뿌리공원 여행 대전.csv
│       ├── 뿌리공원 여행 주말에.csv
│       ├── 뿌리공원 관광 주말에.csv
│       ├── 엑스포과학공원 여행 대전.csv
│       └── 엑스포과학공원 여행 후기.csv
├── filtered
│   ├── google_review
│   │   └── google_expo_sample.csv
│   └── naver_review
└── GoogleReviewPreprocessing.ipynb

6 directories, 20 files


## Data Load 

In [4]:
import os 
import pandas as pd 
import glob

In [5]:
data_path = os.path.join(os.getcwd(), 'data/google_review/')
data_path

'/home/aiffel-dj19/jungcheck/DataPreprocessing/data/google_review/'

In [6]:
files = os.listdir(data_path)
files

['세계엑스포기념품박물관.csv', '엑스포음악분수.csv', '엑스포과학공원.csv', '엑스포다리.csv', '한빛탑.csv']

In [7]:
# 검색어들 기준했을때 target은 expo 
target = 'expo'

In [8]:
# 디렉토리내 csv파일 한 개 파일만 load
expo = pd.read_csv(data_path+os.listdir(data_path)[0], encoding='utf-8')
expo['keyword'] = os.listdir(data_path)[0].split('.')[0] 
expo.head()

,name,ratings,date,comment,keyword
0,SKY RHEE,5,4달 전,없음,세계엑스포기념품박물관
1,유현진,5,1년 전,없음,세계엑스포기념품박물관
2,dm Jung,5,1년 전,"평점 2.9점을 믿을 수가 없다.\n19세기에서 20세기를 지나오며,\n세계 최대의...",세계엑스포기념품박물관
3,박성균,3,1년 전,맥주축제,세계엑스포기념품박물관
4,Алексе́й,5,2년 전,역대 엑스포 개최도시의 기념품과 희귀한 아이템을 한 곳에서 볼 수 있습니다.,세계엑스포기념품박물관


In [9]:
df_list = [] 

for file_name in files: 
    file_path = os.path.join(data_path, file_name) # 파일명으로 path 지정
    keyword = file_name.split('.')[0]              # 파일명에서 키워드만 추출    
    df = pd.read_csv(file_path, encoding='utf-8')  # csv 파일 읽기
    df['keyword'] = keyword
    df['target'] = target
    df_list.append(df)
    
data_df = pd.concat(df_list, axis=0)
data_df

,name,ratings,date,comment,keyword,target
0,SKY RHEE,5,4달 전,없음,세계엑스포기념품박물관,expo
1,유현진,5,1년 전,없음,세계엑스포기념품박물관,expo
2,dm Jung,5,1년 전,"평점 2.9점을 믿을 수가 없다.\n19세기에서 20세기를 지나오며,\n세계 최대의...",세계엑스포기념품박물관,expo
3,박성균,3,1년 전,맥주축제,세계엑스포기념품박물관,expo
4,Алексе́й,5,2년 전,역대 엑스포 개최도시의 기념품과 희귀한 아이템을 한 곳에서 볼 수 있습니다.,세계엑스포기념품박물관,expo
...,...,...,...,...,...,...
805,H.C LEE.,5,3년 전,초딩때보고 성인이 되서 보니 또 감회가 새롭네요,한빛탑,expo
806,Chamintha Jayani,3,3년 전,없음,한빛탑,expo
807,Henseg O,2,3년 전,단층으로 요금내고 올라가기 좀 그래요,한빛탑,expo
808,Jinsuk Kim,5,4년 전,여름에는 음악분수도 예쁘고 공연도 많이 함.,한빛탑,expo


## Data cleaning
* null 처리 
* 일정 길이 이하의 문자열 제거 
* 중복 제거 

In [10]:
# null 처리
print(f'Before length: {len(data_df)}')

data_df = data_df[data_df['comment'] != '없음']
print(f'After length: {len(data_df)}')

Before length: 1812
After length: 813


In [11]:
# 단어 1개인 경우 확인 
data_df[data_df['comment'].map(len) == 1]

,name,ratings,date,comment,keyword,target
5,전유진,5,2일 전,굿,엑스포다리,expo
190,김준선,5,6달 전,굿,엑스포다리,expo
8,노태훈,4,2일 전,굿,한빛탑,expo


데이터를 확인했을때 유의미한 경우에 모두 해당했기 때문에 length=1인 데이터 유지 

In [12]:
# 단어 0개인 comment 제외 
print(f'Before length: {len(data_df)}')

data_df = data_df[data_df['comment'].map(len) > 0] 
print(f'After length: {len(data_df)}')

Before length: 813
After length: 813


In [13]:
# 중복제거 
print(f'Before length: {len(data_df)}')

data_df = data_df.drop_duplicates(['name', 'keyword', 'date']) # name과 keyword, date 모두 중복일때 제거 
print(f'After length: {len(data_df)}')

Before length: 813
After length: 813


In [14]:
data_df

,name,ratings,date,comment,keyword,target
2,dm Jung,5,1년 전,"평점 2.9점을 믿을 수가 없다.\n19세기에서 20세기를 지나오며,\n세계 최대의...",세계엑스포기념품박물관,expo
3,박성균,3,1년 전,맥주축제,세계엑스포기념품박물관,expo
4,Алексе́й,5,2년 전,역대 엑스포 개최도시의 기념품과 희귀한 아이템을 한 곳에서 볼 수 있습니다.,세계엑스포기념품박물관,expo
7,Robert Helvie,3,4년 전,(Google 번역 제공) 남북한에 관한 흥미로운 것들. 거기에서 많은 시간을 보내...,세계엑스포기념품박물관,expo
11,hyung-jun kjun,1,5년 전,엑스포는 없고 너무 지루했어,세계엑스포기념품박물관,expo
...,...,...,...,...,...,...
800,J Park,1,3년 전,돈내고 입장하지 마시오..그냥 밖에서 구경만..,한빛탑,expo
805,H.C LEE.,5,3년 전,초딩때보고 성인이 되서 보니 또 감회가 새롭네요,한빛탑,expo
807,Henseg O,2,3년 전,단층으로 요금내고 올라가기 좀 그래요,한빛탑,expo
808,Jinsuk Kim,5,4년 전,여름에는 음악분수도 예쁘고 공연도 많이 함.,한빛탑,expo


## Data preprocessing

In [15]:
import copy # deep copy

In [16]:
filtered_df = copy.deepcopy(data_df)

데이터셋을 살펴보면, 외국인이 작성한 것도 한글로 번역되어 크롤링됐다.
따로 영어를 살려줄 필요 없어 보이므로 한글, ~숫자~ 제외하고 다 제거한다. 

In [17]:
# filtered_df['comment'] = filtered_df['comment'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
filtered_df['comment'] = filtered_df['comment'].str.replace("[^가-힣 ]","", regex=True) # 완성형 한글만
filtered_df

,name,ratings,date,comment,keyword,target
2,dm Jung,5,1년 전,평점 점을 믿을 수가 없다세기에서 세기를 지나오며세계 최대의 축제였던 엑스포요즘의 ...,세계엑스포기념품박물관,expo
3,박성균,3,1년 전,맥주축제,세계엑스포기념품박물관,expo
4,Алексе́й,5,2년 전,역대 엑스포 개최도시의 기념품과 희귀한 아이템을 한 곳에서 볼 수 있습니다,세계엑스포기념품박물관,expo
7,Robert Helvie,3,4년 전,번역 제공 남북한에 관한 흥미로운 것들 거기에서 많은 시간을 보내지 않아도됩니다 ...,세계엑스포기념품박물관,expo
11,hyung-jun kjun,1,5년 전,엑스포는 없고 너무 지루했어,세계엑스포기념품박물관,expo
...,...,...,...,...,...,...
800,J Park,1,3년 전,돈내고 입장하지 마시오그냥 밖에서 구경만,한빛탑,expo
805,H.C LEE.,5,3년 전,초딩때보고 성인이 되서 보니 또 감회가 새롭네요,한빛탑,expo
807,Henseg O,2,3년 전,단층으로 요금내고 올라가기 좀 그래요,한빛탑,expo
808,Jinsuk Kim,5,4년 전,여름에는 음악분수도 예쁘고 공연도 많이 함,한빛탑,expo


정규표현식을 통해 제거 후에, 문장 구성이 달라지기 때문에 cleaning 작업을 위와 똑같이 다시 진행해줍니다.

In [18]:
# 단어 1개인 경우 확인 
filtered_df[filtered_df['comment'].map(len) == 1]

,name,ratings,date,comment,keyword,target
5,전유진,5,2일 전,굿,엑스포다리,expo
190,김준선,5,6달 전,굿,엑스포다리,expo
204,욧이나묵어라,4,6달 전,굿,엑스포다리,expo
270,김칠성,5,8달 전,음,엑스포다리,expo
8,노태훈,4,2일 전,굿,한빛탑,expo
560,mini star,4,1년 전,굿,한빛탑,expo


데이터를 확인했을때 유의미한 경우에 모두 해당했기 때문에 length=1인 데이터 유지 

In [19]:
# 단어 0개인 comment 제외 
print(f'Before length: {len(filtered_df)}')

filtered_df = filtered_df[filtered_df['comment'].map(len) > 0] 
print(f'After length: {len(filtered_df)}')

Before length: 813
After length: 812


In [20]:
# 중복제거 
print(f'Before length: {len(filtered_df)}')

filtered_df = filtered_df.drop_duplicates(['name', 'keyword', 'date']) # name과 keyword, date 모두 중복일때 제거 
print(f'After length: {len(filtered_df)}')

Before length: 812
After length: 812


In [21]:
filtered_df

,name,ratings,date,comment,keyword,target
2,dm Jung,5,1년 전,평점 점을 믿을 수가 없다세기에서 세기를 지나오며세계 최대의 축제였던 엑스포요즘의 ...,세계엑스포기념품박물관,expo
3,박성균,3,1년 전,맥주축제,세계엑스포기념품박물관,expo
4,Алексе́й,5,2년 전,역대 엑스포 개최도시의 기념품과 희귀한 아이템을 한 곳에서 볼 수 있습니다,세계엑스포기념품박물관,expo
7,Robert Helvie,3,4년 전,번역 제공 남북한에 관한 흥미로운 것들 거기에서 많은 시간을 보내지 않아도됩니다 ...,세계엑스포기념품박물관,expo
11,hyung-jun kjun,1,5년 전,엑스포는 없고 너무 지루했어,세계엑스포기념품박물관,expo
...,...,...,...,...,...,...
800,J Park,1,3년 전,돈내고 입장하지 마시오그냥 밖에서 구경만,한빛탑,expo
805,H.C LEE.,5,3년 전,초딩때보고 성인이 되서 보니 또 감회가 새롭네요,한빛탑,expo
807,Henseg O,2,3년 전,단층으로 요금내고 올라가기 좀 그래요,한빛탑,expo
808,Jinsuk Kim,5,4년 전,여름에는 음악분수도 예쁘고 공연도 많이 함,한빛탑,expo


In [22]:
filtered_path = os.path.join(os.getcwd(), 'filtered/google_review/')
filtered_path

'/home/aiffel-dj19/jungcheck/DataPreprocessing/filtered/google_review/'

In [23]:
save_path = filtered_path + 'google_expo_sample.csv'

In [24]:
filtered_df.to_csv(save_path, header=True, index=False)

In [25]:
ls ./data/google_review

한빛탑.csv      엑스포음악분수.csv  세계엑스포기념품박물관.csv
엑스포다리.csv  엑스포과학공원.csv


In [26]:
# 저장한 csv 재확인
pd.read_csv(save_path)

,name,ratings,date,comment,keyword,target
0,dm Jung,5,1년 전,평점 점을 믿을 수가 없다세기에서 세기를 지나오며세계 최대의 축제였던 엑스포요즘의 ...,세계엑스포기념품박물관,expo
1,박성균,3,1년 전,맥주축제,세계엑스포기념품박물관,expo
2,Алексе́й,5,2년 전,역대 엑스포 개최도시의 기념품과 희귀한 아이템을 한 곳에서 볼 수 있습니다,세계엑스포기념품박물관,expo
3,Robert Helvie,3,4년 전,번역 제공 남북한에 관한 흥미로운 것들 거기에서 많은 시간을 보내지 않아도됩니다 ...,세계엑스포기념품박물관,expo
4,hyung-jun kjun,1,5년 전,엑스포는 없고 너무 지루했어,세계엑스포기념품박물관,expo
...,...,...,...,...,...,...
807,J Park,1,3년 전,돈내고 입장하지 마시오그냥 밖에서 구경만,한빛탑,expo
808,H.C LEE.,5,3년 전,초딩때보고 성인이 되서 보니 또 감회가 새롭네요,한빛탑,expo
809,Henseg O,2,3년 전,단층으로 요금내고 올라가기 좀 그래요,한빛탑,expo
810,Jinsuk Kim,5,4년 전,여름에는 음악분수도 예쁘고 공연도 많이 함,한빛탑,expo
